In [1]:
import awkward as ak
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema
import numpy as np
from tqdm import tqdm,trange
import matplotlib.pyplot as plt
import numba

### Setup parameters

In [2]:
infile = "data/signal/wza_UL18_sum.root" # --signal

#infile = "data/mc/TTZToLL/30FD3469-AEFB-B742-8F3E-4551B23D529B.root"
#infile='data/data/7B56E217-555E-1C41-9494-491849A9835F_skim_2ElIdPt20.root' # --data 
#infile = 'data/data/CA42F3A2-614F-4A4F-AF18-F6E66CDA2C85_skim_2ElIdPt20.root'
#infile = 'data/Ntuple/EGamma_Run2018B.root'
#infile="data/mc/59AB328B-F0E3-F544-98BB-E5E55577C649_skim_2ElIdPt20.root" # --mc

dataset="WZG"
year='2018'

In [3]:
events = NanoEventsFactory.from_root(infile, schemaclass=NanoAODSchema).events()

In [4]:
# Trigger set
doubleelectron_triggers  ={
    '2018': [
            "Ele23_Ele12_CaloIdL_TrackIdL_IsoVL", # Recomended
            ]
}



singleelectron_triggers = { #2017 and 2018 from monojet, applying dedicated trigger weights
        '2016': [
            'Ele27_WPTight_Gsf',
            'Ele105_CaloIdVT_GsfTrkIdT'
        ],
        '2017': [
            'Ele35_WPTight_Gsf',
            'Ele115_CaloIdVT_GsfTrkIdT',
            'Photon200'
        ],
        '2018': [
            'Ele32_WPTight_Gsf',    # Recomended
        ]
    }



isData = "genWeight" not in events.fields


# Golden Json file
if (year == "2018") and isData:
    injson = "data/json/Cert_314472-325175_13TeV_Legacy2018_Collisions18_JSON.txt.RunABD"

### Helper functions

In [5]:
# << Sort by PT  helper function >>
def sort_by_pt(ele,pho,jet):
    ele = ele[ak.argsort(ele.pt,ascending=False,axis=1)]
    pho = pho[ak.argsort(pho.pt,ascending=False,axis=1)]
    jet = jet[ak.argsort(jet.pt,ascending=False,axis=1)]

    return ele,pho,jet
events.Electron, events.Photon, events.Jet = sort_by_pt(
            events.Electron, events.Photon, events.Jet
        )

# Lorentz vectors
from coffea.nanoevents.methods import vector
ak.behavior.update(vector.behavior)

def TLorentz_vector(vec):
    vec = ak.zip(
        {"x": vec.x, "y": vec.y, "z": vec.z, "t": vec.t},
        with_name="LorentzVector",
    )
    return vec

def TLorentz_vector_cylinder(vec):

    vec = ak.zip(
        {
            "pt": vec.pt,
            "eta": vec.eta,
            "phi": vec.phi,
            "mass": vec.mass,
        },
        with_name="PtEtaPhiMLorentzVector",
    )

    return vec



### Good-Run check ( Data-only )

In [6]:
from coffea import lumi_tools

if isData:
    lumi_mask_builder = lumi_tools.LumiMask(injson)
    lumimask = ak.Array(
        lumi_mask_builder.__call__(events.run, events.luminosityBlock)
    )
    events = events[lumimask]

### 1. Trigger

In [7]:
# double lepton trigger
is_double_ele_trigger=True
if not is_double_ele_trigger:
    double_ele_triggers_arr=np.ones(len(events), dtype=np.bool)
else:
    double_ele_triggers_arr = np.zeros(len(events), dtype=np.bool)
    for path in doubleelectron_triggers[year]:
        if path not in events.HLT.fields: continue
        double_ele_triggers_arr = double_ele_triggers_arr | events.HLT[path]


# single lepton trigger
is_single_ele_trigger=True
if not is_single_ele_trigger:
    single_ele_triggers_arr=np.ones(len(events), dtype=np.bool)
else:
    single_ele_triggers_arr = np.zeros(len(events), dtype=np.bool)
    for path in singleelectron_triggers[year]:
        if path not in events.HLT.fields: continue
        single_ele_triggers_arr = single_ele_triggers_arr | events.HLT[path]

# Sort particle order by PT  # RunD --> has problem
events.Electron,events.Photon,events.Jet = sort_by_pt(events.Electron,events.Photon,events.Jet)

Initial_events = events
#events = events[single_ele_triggers_arr | double_ele_triggers_arr]
events = events[double_ele_triggers_arr]
print("Events passing triggers and skiimig: ",len(events) )

cut1 = np.ones(len(events))
# Particle Identification


Events passing triggers and skiimig:  22276


In [8]:
# Set Particle
Electron = events.Electron
Muon = events.Muon
Photon = events.Photon
MET = events.MET
Jet = events.Jet

### 2. Lepton Selection: e e e

In [9]:
#  --Muon ( only used to calculate dR )
MuSelmask = (
    (Muon.pt >= 10)
    & (abs(Muon.eta) <= 2.5)
    & (Muon.tightId)
    & (Muon.pfRelIso04_all < 0.15)
)
# Muon = ak.mask(Muon,MuSelmask)
Muon = Muon[MuSelmask]

In [10]:
##----------- Cut flow2: Electron Selection

EleSelmask = (
    (Electron.pt >= 10)
    & (np.abs(Electron.eta + Electron.deltaEtaSC) < 1.479)
    & (Electron.cutBased > 2)
    & (abs(Electron.dxy) < 0.05)
    & (abs(Electron.dz) < 0.1)
) | (
    (Electron.pt >= 10)
    & (np.abs(Electron.eta + Electron.deltaEtaSC) > 1.479)
    & (np.abs(Electron.eta + Electron.deltaEtaSC) <= 2.5)
    & (Electron.cutBased > 2)
    & (abs(Electron.dxy) < 0.1)
    & (abs(Electron.dz) < 0.2)
)

Electron = Electron[EleSelmask]

In [11]:
# apply cut 2
Tri_electron_mask = ak.num(Electron) == 3
Electron = Electron[Tri_electron_mask]
Photon = Photon[Tri_electron_mask]
Jet = Jet[Tri_electron_mask]
MET = MET[Tri_electron_mask]
Muon = Muon[Tri_electron_mask]
events = events[Tri_electron_mask]

In [12]:
len(events)

2198

### 3. Photon Selection:  pho > 0 

In [13]:
##----------- Cut flow3: Photon Selection

# Basic photon selection
isgap_mask = (abs(Photon.eta) < 1.442) | (
    (abs(Photon.eta) > 1.566) & (abs(Photon.eta) < 2.5)
)
Pixel_seed_mask = ~Photon.pixelSeed

if (dataset == "ZZ") and (self._year == "2017"):
    PT_ID_mask = (Photon.pt >= 20) & (
        Photon.cutBasedBitmap >= 3
    )  # 2^0(Loose) + 2^1(Medium) + 2^2(Tights)
else:
    PT_ID_mask = (Photon.pt >= 20) & (Photon.cutBased > 1)

# dR cut with selected Muon and Electrons
dr_pho_ele_mask = ak.all(
    Photon.metric_table(Electron) >= 0.5, axis=-1
)  # default metric table: delta_r
dr_pho_mu_mask = ak.all(Photon.metric_table(Muon) >= 0.5, axis=-1)

# genPartFlav cut
"""
if dataset == "WZG":
    isPrompt = (Photon.genPartFlav == 1) | (Photon.genPartFlav == 11)
    PhoSelmask = PT_ID_mask & isgap_mask &  Pixel_seed_mask & isPrompt & dr_pho_ele_mask & dr_pho_mu_mask
elif dataset == "WZ":
    isPrompt = (Photon.genPartFlav == 1) 
    PhoSelmask = PT_ID_mask & isgap_mask &  Pixel_seed_mask & ~isPrompt & dr_pho_ele_mask & dr_pho_mu_mask

else:
    PhoSelmask = PT_ID_mask  & isgap_mask &  Pixel_seed_mask & dr_pho_ele_mask & dr_pho_mu_mask
"""

PhoSelmask = (
    PT_ID_mask & isgap_mask & Pixel_seed_mask & dr_pho_ele_mask & dr_pho_mu_mask
)
Photon = Photon[PhoSelmask]

In [14]:
# Apply cut 3
A_photon_mask = ak.num(Photon) > 0
Electron = Electron[A_photon_mask]
Photon = Photon[A_photon_mask]
Jet = Jet[A_photon_mask]
Muon = Muon[A_photon_mask]
MET = MET[A_photon_mask]
events = events[A_photon_mask]

In [15]:
len(events)

588

### 4. OSSF Selection:  Ze1 Ze2 We1

In [16]:
##----------- Cut flow4: OSSF
# OSSF index maker
@numba.njit
def find_3lep(events_leptons, builder):
    for leptons in events_leptons:

        builder.begin_list()
        nlep = len(leptons)
        for i0 in range(nlep):
            for i1 in range(i0 + 1, nlep):
                if leptons[i0].charge + leptons[i1].charge != 0:
                    continue

                for i2 in range(nlep):
                    if len({i0, i1, i2}) < 3:
                        continue
                    builder.begin_tuple(3)
                    builder.index(0).integer(i0)
                    builder.index(1).integer(i1)
                    builder.index(2).integer(i2)
                    builder.end_tuple()
        builder.end_list()
    return builder

eee_triplet_idx = find_3lep(Electron, ak.ArrayBuilder()).snapshot()

ossf_mask = ak.num(eee_triplet_idx) == 2

/home/jkim/miniconda3/envs/pyHEP/lib/python3.8/site-packages/numba/core/dispatcher.py:238: UserWarning: Numba extension module 'awkward1._connect._numba' failed to load due to 'ImportError(generic_type: type "kernel_lib" is already registered!)'.
  entrypoints.init_all()


In [17]:
# Apply cut 4
eee_triplet_idx = eee_triplet_idx[ossf_mask]
Electron = Electron[ossf_mask]
Photon = Photon[ossf_mask]
Jet = Jet[ossf_mask]
MET = MET[ossf_mask]
events = events[ossf_mask]

In [18]:
len(events)

585

In [19]:
# Define Electron Triplet

Triple_electron = [Electron[eee_triplet_idx[idx]] for idx in "012"]
Triple_eee = ak.zip(
    {
        "lep1": Triple_electron[0],
        "lep2": Triple_electron[1],
        "lep3": Triple_electron[2],
        "p4": TLorentz_vector(Triple_electron[0] + Triple_electron[1]),
    }
)

# Ele pair selector --> Close to Z mass
bestZ_idx = ak.singletons(ak.argmin(abs(Triple_eee.p4.mass - 91.1876), axis=1))
Triple_eee = Triple_eee[bestZ_idx]

leading_ele = Triple_eee.lep1
subleading_ele = Triple_eee.lep2
third_ele = Triple_eee.lep3

def make_leading_pair(target, base):
    return target[ak.argmax(base.pt, axis=1, keepdims=True)]

leading_pho = make_leading_pair(Photon, Photon)

### 5. Event selection

In [20]:
diele = Triple_eee.p4
Mee_cut_mask = ak.firsts(diele.mass) > 4

# Electron PT cuts
Elept_mask = ak.firsts(
    (leading_ele.pt >= 25) & (subleading_ele.pt >= 10) & (third_ele.pt >= 25)
)

# MET cuts
MET_mask = MET.pt > 20  # Baseline
# MET_mask = MET.pt > 30 #  SR, CR-ZZE, CR-t-entirched
# MET_mask = MET.pt <= 30 #  CR-Z+jets. CR-Conversion

# Mask
Event_sel_mask = Elept_mask & MET_mask & Mee_cut_mask  # Baseline

In [21]:
# Apply cut5
Triple_eee_sel = Triple_eee[Event_sel_mask]
leading_pho_sel = leading_pho[Event_sel_mask]
MET_sel = MET[Event_sel_mask]
events = events[Event_sel_mask]

In [22]:
len(events)

405

In [23]:
# Photon  EE and EB
isEE_mask = leading_pho.isScEtaEE
isEB_mask = leading_pho.isScEtaEB
Pho_EE = leading_pho[isEE_mask & Event_sel_mask]
Pho_EB = leading_pho[isEB_mask & Event_sel_mask]

### 6. Prepare hist

In [24]:
# Photon
phoPT = ak.flatten(leading_pho_sel.pt)
phoEta = ak.flatten(leading_pho_sel.eta)
phoPhi = ak.flatten(leading_pho_sel.phi)

# Photon EE
if len(Pho_EE.pt) != 0:
    Pho_EE_PT = ak.flatten(Pho_EE.pt)
    Pho_EE_Eta = ak.flatten(Pho_EE.eta)
    Pho_EE_Phi = ak.flatten(Pho_EE.phi)
    Pho_EE_sieie = ak.flatten(Pho_EE.sieie)
    Pho_EE_hoe = ak.flatten(Pho_EE.hoe)
    Pho_EE_Iso_all = ak.flatten(Pho_EE.pfRelIso03_all)
    Pho_EE_Iso_charge = ak.flatten(Pho_EE.pfRelIso03_chg)

# Photon EB
if len(Pho_EB.pt) != 0:
    Pho_EB_PT = ak.flatten(Pho_EB.pt)
    Pho_EB_Eta = ak.flatten(Pho_EB.eta)
    Pho_EB_Phi = ak.flatten(Pho_EB.phi)
    Pho_EB_sieie = ak.flatten(Pho_EB.sieie)
    Pho_EB_hoe = ak.flatten(Pho_EB.hoe)
    Pho_EB_Iso_all = ak.flatten(Pho_EB.pfRelIso03_all)
    Pho_EB_Iso_charge = ak.flatten(Pho_EB.pfRelIso03_chg)

# Electrons
ele1PT = ak.flatten(Triple_eee_sel.lep1.pt)
ele1Eta = ak.flatten(Triple_eee_sel.lep1.eta)
ele1Phi = ak.flatten(Triple_eee_sel.lep1.phi)

ele2PT = ak.flatten(Triple_eee_sel.lep2.pt)
ele2Eta = ak.flatten(Triple_eee_sel.lep2.eta)
ele2Phi = ak.flatten(Triple_eee_sel.lep2.phi)

ele3PT = ak.flatten(Triple_eee_sel.lep3.pt)
ele3Eta = ak.flatten(Triple_eee_sel.lep3.eta)
ele3Phi = ak.flatten(Triple_eee_sel.lep3.phi)

charge = ak.flatten(Triple_eee.lep1.charge + Triple_eee.lep2.charge)

# MET
met = ak.to_numpy(MET_sel.pt)

# M(eea) M(ee)
diele = Triple_eee_sel.p4
eeg_vec = diele + leading_pho_sel
Meea = ak.flatten(eeg_vec.mass)
Mee = ak.flatten(Triple_eee_sel.p4.mass)

# W MT (--> beta )
Ele3 = ak.flatten(Triple_eee_sel.lep3)
MT = np.sqrt(
    2 * Ele3.pt * MET_sel.pt * (1 - np.cos(abs(MET_sel.delta_phi(Ele3))))
)
MT = np.array(MT)